In [128]:
import numpy as np
import pandas as pd
import requests
import urllib

In [105]:
df_logs = pd.read_csv(
    "./list_searches.txt",
    on_bad_lines="skip",
    index_col=None,
    header=None,
    names=["terms"],
)
df_logs.head(3)

,terms
0,Agence Interim Adecco
1,Fondation adecco
2,Club mediterranee


In [106]:
df_logs.shape

(17779, 1)

In [107]:
df_matomo = pd.read_csv("./elastic_vs_postgres_v0.csv")

In [108]:
df_matomo.head(3)

,terms,siren,url,Status,Commentaire,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres
0,stellantis,879786085,https://annuaire-entreprises.data.gouv.fr/entr...,TRUE,542 065 479 bizarre,1353,68,"['879786085', '803902485', '879147148', '34219...",0.065801,1,1,['879786085'],0.029909,1,1,0,0
1,STELLANTIS,879786085,https://annuaire-entreprises.data.gouv.fr/entr...,TRUE,542 065 479 bizarre,1353,68,"['879786085', '803902485', '879147148', '34219...",0.070908,1,1,['879786085'],0.021949,1,1,0,0
2,Stellantis,879786085,https://annuaire-entreprises.data.gouv.fr/entr...,TRUE,542 065 479 bizarre,1353,68,"['879786085', '803902485', '879147148', '34219...",0.080831,1,1,['879786085'],0.022308,1,1,0,0


In [109]:
df_logs

,terms
0,Agence Interim Adecco
1,Fondation adecco
2,Club mediterranee
3,Club med
4,Emmaus communaute
...,...
17774,Climatisation
17775,ACTION IMMOBILIERE EUROPEENNE
17776,TWO ON A BENCH
17777,Michaux stephane


In [110]:
df_matomo["terms"] = df_matomo["terms"].str.lower()
df_logs["terms"] = df_logs["terms"].str.lower()

In [111]:
df_logs.shape

(17779, 1)

In [112]:
count = 0
for term_log in df_logs["terms"]:
    for term_matomo in df_matomo["terms"]:
        if term_log == term_matomo:
            df_logs = df_logs[df_logs["terms"] != term_matomo]
            count += 1

In [113]:
count

1648

In [114]:
df_logs

,terms
0,agence interim adecco
1,fondation adecco
2,club mediterranee
3,club med
4,emmaus communaute
...,...
17774,climatisation
17775,action immobiliere europeenne
17776,two on a bench
17777,michaux stephane


In [115]:
df_logs.drop_duplicates(subset=["terms"], keep="last", inplace=True)

In [116]:
df_logs.shape

(16621, 1)

In [117]:
for term in df_logs["terms"]:
    if term.isdecimal():
        df_logs = df_logs[df_logs["terms"] != term]

In [118]:
df_logs.shape

(16079, 1)

In [120]:
df_logs["url_post"] = ""
df_logs["url_elastic"] = ""

In [121]:
df_logs.shape

(16079, 3)

In [135]:
for index, row in df_logs.iterrows():
    terms = df_logs["terms"][index]
    df_logs["url_post"][
        index
    ] = f'https://annuaire-entreprises.data.gouv.fr/rechercher?terme={urllib.parse.quote(terms, safe="")}'
    df_logs["url_elastic"][
        index
    ] = f'https://test.annuaire-entreprises.data.gouv.fr/rechercher?terme={urllib.parse.quote(terms, safe="")}'

In [123]:
df_logs.head(5)

,terms,url_post,url_elastic
0,agence interim adecco,https://annuaire-entreprises.data.gouv.fr/rech...,https://test.annuaire-entreprises.data.gouv.fr...
1,fondation adecco,https://annuaire-entreprises.data.gouv.fr/rech...,https://test.annuaire-entreprises.data.gouv.fr...
2,club mediterranee,https://annuaire-entreprises.data.gouv.fr/rech...,https://test.annuaire-entreprises.data.gouv.fr...
4,emmaus communaute,https://annuaire-entreprises.data.gouv.fr/rech...,https://test.annuaire-entreprises.data.gouv.fr...
7,psa peugeot,https://annuaire-entreprises.data.gouv.fr/rech...,https://test.annuaire-entreprises.data.gouv.fr...


In [138]:
df_logs.to_csv("./dataset_logs_ngnix.csv", header=True, index=False)

In [137]:
df_logs["url_post"][1]

'https://annuaire-entreprises.data.gouv.fr/rechercher?terme=fondation%20adecco'